In [3]:
from itertools import combinations
import numpy as np

In [7]:
np.dot([1, 0], [0,1])

0

In [10]:
np.arange(len(np.array([1, 2, 3])))[np.array([1, 2, 3])==np.array([1, 2, 3])]

array([0, 1, 2])

In [14]:
np.apply_along_axis(lambda x: x+1, 0, np.array([1,2,3,4,5,6]))

array([2, 3, 4, 5, 6, 7])